# D2E2F: Helsingborg-Helsingør
(2021-04-23)

In [ ]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
plt.rcParams["figure.figsize"] = (20,3)
plt.style.use('presentation')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize
from src.data import get_dataset
from src.data import prepare_dataset
from src.data import trips
import scipy.integrate
import seaborn as sns
from ipywidgets import Layout, interact, IntSlider

In [ ]:
df = get_dataset.get(name='tycho_short_id')

## Plot trips

In [ ]:
visualize.plot_map(df=df, width=1400, height=800, zoom_start=14)

## Trip statistics

In [ ]:
df_stat = get_dataset.trip_statistics()

In [ ]:
df_stat_plot = df_stat.copy()

df_stat_plot['energy per trip [Nm]'] = df_stat_plot['power_em_thruster_total']

df_stat_plot['trip_direction'] = df_stat_plot['trip_direction'].apply(lambda x : 'Helsingør-Helsingborg' if x==0 else 'Helsingborg-Helsingør')

fig = px.scatter(df_stat_plot, x='start_time',y='energy per trip [Nm]', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'])
fig.show()

## Analyzing trips

In [ ]:
fig = px.line(df, x='trip_time', y='sog', color='trip_no', width=1500, height=800)
fig.show()

In [ ]:
def plot_thruster(ax, x,y,cos,sin, power, scale=20):
    
    l = power*scale
    ax.arrow(x=x, y=y, dx=-l*sin, dy=-l*cos, head_width=l/5, head_length=l/5)
    
def plot_thrust_allocation(row, lpp=50, beam=20, scale=30):
    
    #fig,axes=plt.subplots(ncols=2)
    fig = plt.figure(constrained_layout=True)
    fig.set_size_inches(20,10)
    
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[:, 0])
    # identical to ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1:, 1])
    
    ax = ax1    
    positions = np.array([
        [-1,-1],
        [1,-1],
        [1,1],
        [-1,1],
        
    ])
    
    coordinates = np.array([positions[:,0]*beam/2, positions[:,1]*lpp/2] ).T
        
    
    for i,position in enumerate(coordinates):
        
        n=i+1
        sin_key = 'sin_pm%i' % n
        cos_key = 'cos_pm%i' % n
        power_key = 'power_em_thruster_%i' % n
                
        plot_thruster(ax=ax, x=position[0], y=position[1], cos=row[cos_key], sin=row[sin_key], power=row[power_key], scale=scale)
        
    
    ax.set_xlim(np.min(coordinates[:,0])-scale,np.max(coordinates[:,0])+scale)
    ax.set_ylim(np.min(coordinates[:,1])-scale,np.max(coordinates[:,1])+scale)
    #ax.set_aspect('equal', 'box')
    
    ## Second plot:
    ax = ax2
    trip_.plot(x='longitude', y='latitude', ax=ax, style='k--')
    ax.plot(row['longitude'], row['latitude'], 'o', ms=15)
    
    ax.set_ylim(trip_['latitude'].min(), trip_['latitude'].max())
    ax.set_xlim(trip_['longitude'].min(), trip_['longitude'].max())
    #ax.set_aspect('equal')
    
    ## Third plot:
    trip_.plot(x='trip_time_s', y='sog', ax=ax3, style='k--')
    ax3.plot(row['trip_time_s'], row['sog'], 'o', ms=15)
    ax3.set_xlabel('Time [s]')
    ax3.set_ylabel('Ship speed [m/s]')
   
def get_animation(trip_):
    def animate(i=0):
        
        index = int(i)
        row = trip_.iloc[index]
        plot_thrust_allocation(row=row)
        
    return animate
         

## Load trip no. 3

In [ ]:
trips = df.groupby(by='trip_no')
trip = trips.get_group(3)

trip_ = trip.copy()
trip_ = prepare_dataset.calculate_rudder_angles(df=trip_, drop=False)
trip_['trip_time_s'] = pd.TimedeltaIndex(trip_['trip_time']).total_seconds()

power_columns = ['power_em_thruster_%i' % i for i in range(1,5)]
trip_[power_columns]/=trip_['power_em_thruster_total'].max()/4
trip_3 = trip_.copy()

In [ ]:
visualize.plot_map(df=trip_3, width=1400, height=800, zoom_start=14)

In [ ]:
aminate_1 = get_animation(trip_3)
interact(aminate_1, i = IntSlider(0,0,len(trip_)-1,1, layout=Layout(width='100%')));   

## Load trip no. 110

In [ ]:
trips = df.groupby(by='trip_no')
trip = trips.get_group(110)

trip_ = trip.copy()
trip_ = prepare_dataset.calculate_rudder_angles(df=trip_, drop=False)
trip_['trip_time_s'] = pd.TimedeltaIndex(trip_['trip_time']).total_seconds()

power_columns = ['power_em_thruster_%i' % i for i in range(1,5)]
trip_[power_columns]/=trip_['power_em_thruster_total'].max()/4
trip_110 = trip_.copy()

In [ ]:
visualize.plot_map(df=trip_110, width=1400, height=800, zoom_start=14)

In [ ]:
aminate_1 = get_animation(trip_110)
interact(aminate_1, i = IntSlider(0,0,len(trip_)-1,1, layout=Layout(width='100%')));   

## Is there a more optimal thruster operation?

![](https://memegenerator.net/img/images/15921176.jpg)